## Setup autoreload, warnings and helper functions

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, Markdown
def print_heading(string):
    display(Markdown(f"# {string}"))
def print_subheading(string):
    display(Markdown(f"## {string}"))

## Set the visibility of cuda devices (in case your system contains more than one)

In [2]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


## Imports

In [3]:
%matplotlib inline
from config import config
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import zipfile
import tensorflow as tf
import wget
import os
import shutil
import pickle
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from torchvision.datasets.utils import download_url
from datasets import SimpleDataset

from tools import apply_pipeline, crop_step, curry, apply_pipeline_cocodataset, get_save_step
from tonemapping.tonemapping import tonemap, tonemap_step
from segmentation.segmentation import segment
from pattern_extraction.extract_pattern import extract_pattern
from reidentification.identify import encode_single, encode_pipeline, create_database, identify, identify_single, encode_dataset
from reidentification.visualisation import visualise_match
from simple_database import DBDatabase

/ekaterina/env/norppa/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-23 22:46:19.081949: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-02-23 22:46:20.659485: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-02-23 22:46:20.697623: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2023-02-23 22:46:20.697658: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-02-23 22:46:20

## Create a configuration file.
You can change the default parameters in config.py

In [4]:
cfg = config()

/ekaterina/work/src/NORPPA/repository/NORPPA/reidentification/hesaff_pytorch/architectures.py:71: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  nn.init.orthogonal(m.weight.data, gain=0.9)
/ekaterina/work/src/NORPPA/repository/NORPPA/reidentification/hesaff_pytorch/architectures.py:73: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias.data, 0.01)
2023-02-23 22:46:26.899633: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2023-02-23 22:46:26.900407: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2023-02-23 22:46:26.900504: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023

## Path to the dataset

In [5]:


datafolder_dir = "/ekaterina/work/data/sides_dataset"

dataset_dir = Path(datafolder_dir)

print(f'SealID dataset is ready')

SealID dataset is ready


## Create dataset variables

In [6]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.optim import Adam
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
BATCH_SIZE = 12
TRANSFORM_IMG = transforms.Compose([
    transforms.Resize([256,256]),
#     transforms.Grayscale(num_output_channels=1),
#     transforms.CenterCrop(256),
    transforms.ColorJitter(),
    transforms.RandomAffine(30, scale=[0.8, 1.2]),
    transforms.ToTensor(),
    
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225] )
    transforms.Normalize(mean=[0.485],
                         std=[0.229] )
    ])

data = torchvision.datasets.ImageFolder(root=datafolder_dir, transform=TRANSFORM_IMG)
train_dataset, test_dataset = torch.utils.data.random_split(data, [int(0.7 * len(data)), len(data) - int(0.7 * len(data))])


# train_dataset = torch.utils.data.Subset(data, range(1500))  
# val_dataset = torch.utils.data.Subset(data, range(1201, 1500))     
# test_dataset = torch.utils.data.Subset(data, range(1501, 2080))     
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,  num_workers=0)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0) 

# test_data = torchvision.datasets.ImageFolder(root=datafolder_dir, transform=TRANSFORM_IMG)
# test_data_loader  = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4) 

# # Data Loader (Input Pipeline)
# train_loader = torch.utils.data.DataLoader(dataset=TRAIN_DATA_PATH,
#                                            batch_size=batchsize,
#                                            shuffle=True)
# test_loader = torch.utils.data.DataLoader(dataset=TEST_DATA_PATH,
#                                           batch_size=batchsize,
#                                           shuffle=False)


In [7]:
from coatnet.coatnet import CoAtNet,coatnet_0, coatnet_1, coatnet_2, coatnet_3, coatnet_norppa

num_blocks = [2, 2, 3, 5, 2]            # L
channels = [64, 96, 192, 384, 768]      # D
block_types=['C', 'C', 'T', 'T']        # 'C' for MBConv, 'T' for Transformer

# model = CoAtNet((256, 256), 3, num_blocks, channels, block_types=block_types, num_classes=2)
model = coatnet_norppa()
model.load_state_dict(torch.load("./coatnet_bigdata.pth"))
print(model)
# out = net(img)

CoAtNet(
  (s0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): GELU()
    )
  )
  (s1): Sequential(
    (0): MBConv(
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (proj): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (conv): PreNorm(
        (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (fn): Sequential(
          (0): Conv2d(32, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tru

/ekaterina/env/norppa/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
from torch.autograd import Variable
from torch.optim.lr_scheduler import ExponentialLR

torch.set_grad_enabled(True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



# Function to save the model
def saveModel():
    path = "./coatnet_bigdata.pth"
    torch.save(model.state_dict(), path)

# Function to test the model with the test dataset and print the accuracy for the test images
def testAccuracy():
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            # run the model on the test set to predict labels
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy, loss.item())

loss_fn = nn.CrossEntropyLoss()
# Training function. We simply have to loop over our data iterator and feed the inputs to the network and optimize.
def train(num_epochs):
    
    best_accuracy = 0.0
    train_accuracy = 0.0
    total = 0.0
    

    # Define your execution device
    print("The model will be running on", device, "device")
    # Convert model parameters and buffers to CPU or Cuda
    model.to(device)
    
    optimizer = Adam(model.parameters(), lr=0.0001)
    scheduler = ExponentialLR(optimizer, gamma=0.9)
    print_step = 10
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        running_acc = 0.0

        for i, (images, labels) in enumerate(train_loader, 0):
            
            # get the inputs
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            # predict classes using images from the training set
            outputs = model(images)
            # compute the loss based on model output and real labels
            loss = loss_fn(outputs, labels)
            # zero the parameter gradients
            optimizer.zero_grad()
            # backpropagate the loss
            loss.backward()
            # adjust parameters based on the calculated gradients
            optimizer.step()

            running_loss += loss.item()     # extract the loss value
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            train_accuracy += (predicted == labels).sum().item()
            
            if (i+1) % print_step == 0:    
                # print every 1000 (twice per epoch) 
                print('Train accuracy is %d %%' % (100 * train_accuracy / total))
                print('[%d, %5d] loss: %.8f' %
                      (epoch + 1, i + 1, running_loss / print_step))
                # zero the loss
                running_loss = 0.0
        scheduler.step()
        # Compute and print the average accuracy fo this epoch when tested over all 10000 test images
        accuracy, val_loss = testAccuracy()
        print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
        print('For epoch', epoch+1,'the validation loss is %.8f %%' % (val_loss))
        
        # we want to save the model if the accuracy is the best
        if accuracy > best_accuracy:
            saveModel()
            best_accuracy = accuracy

In [9]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
classes = ['left', 'right']

# Function to show the images
def imageshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.figure(figsize = (500,200))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# Function to test the model with a batch of images and show the labels predictions
def testBatch():
    # get batch of images from the test DataLoader  
    images, labels = next(iter(test_loader))

    # show all images as one image grid
    imageshow(torchvision.utils.make_grid(images))
   
    # Show the real labels on the screen 
    print('Real labels: ', ' '.join('%5s' % classes[labels[j]] 
                               for j in range(BATCH_SIZE)))
  
    # Let's see what if the model identifiers the  labels of those example
    outputs = model(images)
    
    # We got the probability for every 10 labels. The highest (max) probability should be correct label
    _, predicted = torch.max(outputs, 1)
    # Let's show the predicted labels on the screen to compare with the real ones
    print('  Predicted: ', ' '.join('%5s' % classes[predicted[j]] 
                              for j in range(BATCH_SIZE)))
    print('Accuracy: ', np.mean(np.array(labels)==np.array(predicted))) 

In [10]:
# Function to test what classes performed well
def testClassess():
    class_correct = list(0. for i in range(2))
    class_total = list(0. for i in range(2))
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(BATCH_SIZE):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(number_of_labels):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

In [11]:
train(300)
print('Finished Training')


The model will be running on cuda:0 device
Train accuracy is 62 %
[1,    10] loss: 0.78176854
Train accuracy is 61 %
[1,    20] loss: 0.67251505
Train accuracy is 60 %
[1,    30] loss: 0.69947605
Train accuracy is 61 %
[1,    40] loss: 0.58492999
Train accuracy is 64 %
[1,    50] loss: 0.54198989
Train accuracy is 65 %
[1,    60] loss: 0.57339353
Train accuracy is 66 %
[1,    70] loss: 0.52315152
Train accuracy is 67 %
[1,    80] loss: 0.58789520
Train accuracy is 68 %
[1,    90] loss: 0.53317778
Train accuracy is 69 %
[1,   100] loss: 0.49732545
Train accuracy is 70 %
[1,   110] loss: 0.51553418
Train accuracy is 70 %
[1,   120] loss: 0.51932682
Train accuracy is 70 %
[1,   130] loss: 0.52403863
Train accuracy is 71 %
[1,   140] loss: 0.49157159
Train accuracy is 70 %
[1,   150] loss: 0.66397982
Train accuracy is 70 %
[1,   160] loss: 0.56686630
Train accuracy is 70 %
[1,   170] loss: 0.61367334
Train accuracy is 71 %
[1,   180] loss: 0.45225234
Train accuracy is 71 %
[1,   190] loss:

In [71]:
train(100)
print('Finished Training')


The model will be running on cuda:0 device
Train accuracy is 69 %
[1,    10] loss: 0.67238414
Train accuracy is 73 %
[1,    20] loss: 0.53794231
Train accuracy is 76 %
[1,    30] loss: 0.45670183
Train accuracy is 75 %
[1,    40] loss: 0.51032148
Train accuracy is 76 %
[1,    50] loss: 0.50763930
Train accuracy is 76 %
[1,    60] loss: 0.45396332
Train accuracy is 77 %
[1,    70] loss: 0.46297933
Train accuracy is 77 %
[1,    80] loss: 0.41418021
Train accuracy is 77 %
[1,    90] loss: 0.53113986
Train accuracy is 77 %
[1,   100] loss: 0.50531890
Train accuracy is 77 %
[1,   110] loss: 0.43886668
Train accuracy is 77 %
[1,   120] loss: 0.43724755
For epoch 1 the test accuracy over the whole test set is 67 %
For epoch 1 the validation loss is 0.90217841 %
Train accuracy is 77 %
[2,    10] loss: 0.43443580
Train accuracy is 77 %
[2,    20] loss: 0.38792761
Train accuracy is 77 %
[2,    30] loss: 0.50287738
Train accuracy is 77 %
[2,    40] loss: 0.47434827
Train accuracy is 76 %
[2,    5

In [14]:
model.to('cpu')

testBatch()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Real labels:  right  left right  left  left right right  left right right  left right
  Predicted:  right  left  left  left  left  left  left  left  left  left right right
Accuracy:  0.5
